# Introduction to LLM Agents with LangChain

## Notebook 1: Tools
---

Welcome to the workshop on building LLM agents with LangChain!

**The goal:** 

With this notebook you will familiarize yourself with the key concepts of Tools as building blocks of an LLM Agent. At the end, you will have all the code you need to use custom LangChain tools as well as build your own custom tools.

🌟 So ... let us begin!  

**The use case:** 

Summer holidays are coming up and you still don't know where to go. Oh no! 

You decide to build a tool that helps you get information on holiday locations. For example, you would like to to find out how big a specific city is, what sights are there to see, how the weather there is, and you would like to get a drawing of that place, to get a first impression. Because who does not like art? 

You will implement this through an LLM agent, who has access to
* the wikipedia API,
* a weather API,
* can generate images by using a HuggingFace API. 

**Content:**

1. [Default LangChain tools](#1)
        <ol type="a">
        <li>[Exercise 1 (a): Explore tool parameters](#1a)</li>
        <li>[Exercise 1 (b): Run tool and explore output](#1b)</li>
        </ol>
2. [Custom tools](#2)
        <ol type="a">
        <li>[Exercise 2 (a): Build your own Weather tool](#2a)</li>
        <li>[Exercise 2 (b): Build your own Image tool](#2b)</li>
        </ol>


**Reminder:** Make sure to update `helper_functions/keys.py` based on keys in [privatebin](https://privatebin.molops.io/?a6459e88fa282c28#DsFZvkZSiuPcNQzNXvmtvmTozihhaf1hQdqBCd7r3q5s)

In [ ]:
# ONLY IF YOU USE GOOGLE COLAB: copy code from helper_functions/colab_utils.txt here and compile

In [2]:
import requests
from helper_functions.keys import WEATHER_KEY, HUGGING_FACE_KEY
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool
from PIL import Image
import io

---

## A Langchain Default tool
### Default [Wikipedia tool](https://python.langchain.com/v0.1/docs/integrations/tools/wikipedia/) <a id='1'></a>

The cell below loads the full wikipedia tool. It makes an API call to Wikipedia using the ``WikipediaAPIWrapper`` and returns a summary of the queried article. ``WikipediaQueryRun`` then wraps this into a ready made tool. 

Each tool is a ``BaseTool`` class object, you can find its definition [here](https://api.python.langchain.com/en/latest/tools/langchain_core.tools.BaseTool.html#langchain_core.tools.BaseTool).

In [ ]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1)
wiki_tool = WikipediaQueryRun(api_wrapper=api_wrapper)

#### Exercise 1 (a): Explore tool parameters <a id='1a'></a>

**TASK:**  
Use the methods ``name``, ``description``, ``args``, ``return_direct``, ``metadata`` to familiarize yourself with the parameters of the tool. What is the meaning of the different parameters?

In [ ]:
print("Name: ", wiki_tool.name)

# TODO: insert your code here

#### Exercise 1 (b): Run tool and explore output <a id='1b'></a>

**TASK:**
* Use the ``.run(tool_input)`` method to execute the tool. The ``tool_input`` is the search term that you'd like to query wikipedia with.
* [Optional] Check out the arguments of the WikipediaAPIWrapper [here](https://api.python.langchain.com/en/latest/utilities/langchain_community.utilities.wikipedia.WikipediaAPIWrapper.html) and modify its parameters above. How does the output change? 

In [ ]:
tool_input = """
TODO: insert your code here
"""

# Run tool
# TODO: insert your code here

---

## Custom tools <a id='2'></a>
### Custom Wikipedia tool

You can build your own tools and don't have to rely on default tools. Tools can be built from any function with the LangChain class method ``StructuredTool.from_function()``(see [here](https://python.langchain.com/v0.1/docs/modules/tools/custom_tools/#structuredtool-dataclass)). The basic elements are
* The **function** you would like to be executed when the tool is called
* The definition of the **input parameters**
* The tool **description**

The tool description is especially important, since this is what the agent will use to make the decision if this tool should be used.

Below you see the wikipedia tool, built from the basic elements described above:

In [ ]:
# define the function
def wikipedia_caller(query:str) ->str:
    """This function queries wikipedia through a search query."""
    return api_wrapper.run(query)

# Input parameter definition
class QueryInput(BaseModel):
    query: str = Field(description="Input search query")

# the tool description
description: str = (
        "A wrapper around Wikipedia. "
        "Useful for when you need to answer general questions about "
        "people, places, companies, facts, historical events, or other subjects. "
        "Input should be a search query."
    )


# fuse the function, input parameters and description into a tool. 
my_own_wiki_tool = StructuredTool.from_function(
    func=wikipedia_caller,
    name="wikipedia",
    description=description,
    args_schema=QueryInput,
    return_direct=False,
)

# test the output of the tool
print(my_own_wiki_tool.run('pyladies'))

#### Exercise 2 (a): Build your own Weather tool <a id='2a'></a>
The goal is to build a tool that extracts weather information from the weather site visualcrossing.com. You typically need an API key to extract information from a website. In this example we provide you with the API key. 

**TASK:** 
- Build the tool by defining the input parameters and the descriptions. The tool function is already provided to you. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``.
- Test if the tool gives an output.

In [ ]:
# define the function
def extract_city_weather(city:str)->str:

    # Build the API URL
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/{city}?key={WEATHER_KEY}&unitGroup=metric"

    response = requests.get(url)

    # extract response
    if response.status_code == 200:
        data = response.json()
        current_temp = data['days'][0]['temp']
        output = f"Current temperature in {city}: {current_temp}°C"
    else:
        output = f"Error: {response.status_code}"

    return output

# Input parameter definition
class WeatherInput(BaseModel):
    # insert your code here

# the tool description
description: str = (
        # TODO: insert your code here
    )

# fuse the function, input parameters and description into a tool. 
my_weather_tool = StructuredTool.from_function(
    # TODO: insert your code here
)

In [ ]:
# Test the output of your Tool
print(my_weather_tool.run('Amsterdam'))

# TODO: Try generating more ouputs

---

Let's do something even more fun. As we previously saw, we can utilize APIs to build tools. Thinking about APIs, one of the biggest collection of models are available via APIs on HuggingFace. So, how about we try to utilize this. 

#### Exercise 2 (b): Build your own Image tool <a id='2b'></a>
The goal is to build a tool that generates an image based on a given prompt. **That means that later when you can build the Agent you can have an LLM that not only outputs text, but also images!**  

To develop this, you can make use of `mobius`, text-to-image model available on HuggingFace. We provided a HuggingFace token (that you loaded in the start). 

**TASK:** 
- Build the tool by defining the input parameters and the descriptions. The tool function is already provided to you. 
- Turn function, description and input parameters into a tool through ``StructuredTool.from_function()``.
- Test if the tool gives an output.

In [ ]:
def text_to_image(payload:str):

    # Call the text-to-image API with the provided palaod
    API_URL = "https://api-inference.huggingface.co/models/Corcelio/mobius"
    headers = {"Authorization": f"Bearer {HUGGING_FACE_KEY}"}

    def query(payload):
        response = requests.post(API_URL, headers=headers, json=payload)
        return response.content
    
    image_bytes = query({
        "inputs": payload,
    })

    image = Image.open(io.BytesIO(image_bytes))
    
    # Resize the image
    new_size = (400, 400)  # Example new size (width, height)
    resized_image = image.resize(new_size)


    # Save the resized image to a file
    image_path = f'images/image_{payload.replace(" ", "_")}.jpg'
    resized_image.save(image_path)
    
    # Return the path to the saved image
    return f'{image_path} '


# Input parameter definition
class ImageInput(BaseModel):
    payload: str = Field(description=
        # TODO: insert your code here
    )


# the tool description
images_description: str = (
       # TODO: insert your code here
    )

# fuse the function, input parameters and description into a tool. 
my_image_tool = StructuredTool.from_function(
        # TODO: insert your code here
)

In [ ]:
# Test the output of your Tool
print(my_image_tool.run('Amsterdam'))

# TODO: Try generating more ouputs

---

**If you developed an additional tool, make sure to copy your code in `helper_functions/tools.py` in order to later be able to use your tool in an Agent.**

Combine the individual tools into a list. Your collection of tools is now ready to be used by an agent.

In [ ]:
tools = [my_own_wiki_tool, my_weather_tool, my_image_tool]

---

🌟 Done - you are now ready to proceed to the next notebook.